# Steps to reproduce experiments in :
## **Reproducible evaluation of methods for predicting progression to Alzheimer’s disease from clinical and neuroimaging data**

## Prerequisites

The original [ADNI](http://adni.loni.usc.edu/) dataset should be downloaded without further touch (Data we used in our paper was downloaded in October 2016).
Fix the paths where your data is stored on your computer:

In [3]:
!export ADNI_PATH="~/Aramis/Data/ADNI"

!export OUT_PATH="~/Aramis/Data/OUTPUT"

!export WORKING_DIR="~/Aramis/Data/tmp/WORKING_DIR"

## 1. Convert datasets into BIDS format

In [ ]:
!clinica convert adni-to-bids $ADNI_PATH/IMAGES $ADNI_PATH/CLINICAL_DATA $OUT_PATH/ADNI/BIDS -m T1 PET_FDG

Define folders for the next steps:

In [ ]:
import os

adnimerge = 'PATH/TO/ADNIMERGE.csv'

adni_bids = os.path.join(os.environ.get('OUT_PATH'), 'ADNI/BIDS')
adni_tsv_dir = os.path.join(os.environ.get('OUT_PATH'), 'ADNI/TSV')
adni_caps_dir = os.path.join(os.environ.get('OUT_PATH'), 'ADNI/CAPS')
adni_output_dir = os.path.join(os.environ.get('OUT_PATH'), 'ADNI/OUTPUT')

working_dir = os.environ.get('WORKING_DIR')


## 2. Create the subjects lists
Choose the subjects at baseline with available T1 MRI for ADNI, AIBL and OASIS, and with FDG-PET for ADNI:

In [ ]:
from Code.subjects_lists.subject_lists import run_subjects_lists

### ADNI dataset
database = 'ADNI'

# For T1
subjects_list = 'T1'
run_subjects_lists(adni_bids, adni_tsv_dir, database, subjects_list, adnimerge)

# For FDG-PET
subjects_list = 'PET'
run_subjects_lists(adni_bids, adni_tsv_dir, database, subjects_list, adnimerge)


## 3. Create demographic tables information
Get demographic information of the different populations:

In [ ]:
from Code.subjects_lists.lists_stats import run_lists_stats

### ADNI dataset
database = 'ADNI'

# For T1
subjects_list = 'T1'
run_lists_stats(adni_bids, adni_tsv_dir, database, subjects_list, adnimerge)

# For FDG-PET
subjects_list = 'PET'
run_lists_stats(adni_bids, adni_tsv_dir, database, subjects_list, adnimerge)


## 4. Run image processing pipelines
Used pipelines are integrated into Clinica software

In [ ]:
### ADNI T1
!clinica run t1-spm-full-prep $OUT_PATH/ADNI/BIDS $OUT_PATH/ADNI/CAPS/ ADNIbl -tsv /SUBJECTS_DIR/subjects_T1_PET.tsv -wd $WORKING_DIR -np 8
### ADNI FDG-PET
!clinica run pet-preprocess-volume $OUT_PATH/ADNI/BIDS $OUT_PATH/ADNI/CAPS/ ADNIbl -tsv $OUT_PATH/ADNI/TSV/subjects_T1_PET.tsv -wd $WORKING_DIR -np 8


In [ ]:
### AIBL and OASIS are registered into ADNI template to later test ADNI trained classifiers on AIBL and OASIS

###AIBL
!clinica run t1-spm-dartel-existing-template $OUT_PATH/AIBL/BIDS $OUT_PATH/AIBL/CAPS/ ADNIbl -tsv $OUT_PATH/AIBL/TSV/subjects_T1.tsv -wd $WORKING_DIR -np 8
!clinica run t1-spm-dartel2mni $OUT_PATH/AIBL/BIDS $OUT_PATH/AIBL/CAPS/ AIBLbl -tsv $OUT_PATH/AIBL/TSV/subjects_T1.tsv -wd $WORKING_DIR -np 8

###OASIS
!clinica run t1-spm-dartel-existing-template $OUT_PATH/OASIS/BIDS $OUT_PATH/OASIS/CAPS/ ADNIbl -tsv $OUT_PATH/OASIS/SUBJECTS_DIR/subjects_T1.tsv -wd $WORKING_DIR -np 8
!clinica run t1-spm-dartel2mni $OUT_PATH/OASIS/BIDS $OUT_PATH/OASIS/CAPS/ OASISbl -tsv $OUT_PATH/OASIS/SUBJECTS_DIR/subjects_T1.tsv -wd $WORKING_DIR -np 8


## 5. Run the SVM classification tasks on imaging data

### Classification results using T1-weighted MRI and FDG-PET images from ADNI dataset

(from `experiments/0-imaging/svm_imaging.py`)

In [ ]:
import os
from os import path
import pandas as pd
import clinica.pipeline.machine_learning.ml_workflows as ml_wf

n_iterations = 250
n_threads = 8

group_id = 'ADNIbl'
tasks = [('sMCI', 'pMCI'),
         ('CN-', 'AD+')]
image_types = ['T1', 'fdg']
fwhm = 4
pvc = None

###################################
### Voxel based classifications ###
###################################
             
for task in tasks:
    subjects_visits_tsv = path.join(adni_tsv_dir, '%s_vs_%s_subjects_sessions.tsv' % (task[0], task[1]))
    diagnoses_tsv = path.join(adni_tsv_dir, '%s_vs_%s_diagnoses.tsv' % (task[0], task[1]))

    for image_type in image_types:
    
        classification_dir = path.join(adni_output_dir, image_type, 'voxel_based', 'linear_svm',
                                       '%s_vs_%s' % (task[0], task[1]))
        if not path.exists(classification_dir):
            os.makedirs(classification_dir)

        print("Running %s" % classification_dir)

        wf = ml_wf.VB_RepHoldOut_DualSVM(adni_caps_dir, subjects_visits_tsv, diagnoses_tsv, group_id, image_type,
                                         classification_dir, fwhm=fwhm, pvc=pvc, n_iterations=n_iterations,
                                         n_threads=n_threads)
        wf.run()


### Applying SVM classifiers trained on CN-AB- vs AD-AB+ task to predict MCI progression

(from `experiments/0-imaging/svm_predict_progression.py`)

In [ ]:
import os
from os import path

import pandas as pd
import numpy as np
import nibabel as nib

import clinica.pipelines.machine_learning.svm_utils as utils

import workflow

spie_output_dir = path.join(adni_output_dir, 'SPIE')
scores_dir = path.join(spie_output_dir, 'input_data', 'svm_scores')

if not path.exists(scores_dir):
            os.makedirs(scores_dir)

group_id = 'ADNIbl'
image_types = ['T1', 'fdg']

tasks = [('sMCI', 'pMCI')]

###################################
### Voxel based classifications ###
###################################

for task in tasks:
    subjects_visits_tsv = path.join(adni_tsv_dir, '%s_vs_%s_subjects_sessions.tsv' % (task[0], task[1]))
    diagnoses_tsv = path.join(adni_tsv_dir, '%s_vs_%s_diagnoses.tsv' % (task[0], task[1]))

    subjects_visits = pd.io.parsers.read_csv(subjects_visits_tsv, sep='\t')
    diagnoses_df = pd.io.parsers.read_csv(diagnoses_tsv, sep='\t')
    
    for image_type in image_types:

        svm_output_dir = path.join(adni_output_dir, image_type, 'voxel_based/linear_svm/CN-_vs_AD+/classifier')
        input_images = workflow.VBAgeCorrectedInput(adni_caps_dir, subjects_visits_tsv, diagnoses_tsv, image_type, None)

        x = input_images.get_x()
        y = input_images.get_y()

        w = np.loadtxt(path.join(svm_output_dir, 'weights.txt'))
        b = np.loadtxt(path.join(svm_output_dir, 'intersect.txt'))

        y_hat = np.dot(w, x.transpose()) + b
        y_binary = (y_hat > 0) * 1.0
        correct = (y == y_binary) * 1.0

        print(utils.evaluate_prediction(y_binary, y))

        results_df = pd.DataFrame({'participant_id': subjects_visits.participant_id.tolist(),
                                   'session_id': subjects_visits.session_id.tolist(),
                                   'diagnosis': diagnoses_df.diagnosis.tolist(),
                                   'y': y.tolist(),
                                   'y_binary': y_binary.tolist(),
                                   'y_hat': y_hat,
                                   'correct': correct})

        results_df.to_csv(path.join(scores_dir, image_type + '.tsv'),
                          index=False, sep='\t', encoding='utf-8')


## 6. Preparing data

Data from different sources (clinical data from BIDS, including ADNIMERGE data, and SVM scores are joined into one file (`input_data.tsv`)
(from `data_selection/join_input_data.py`)

In [ ]:
import pandas as pd
from os import path, environ


adni_bids = path.join(environ.get('OUT_PATH'), 'ADNI/BIDS')
adni_output_dir = path.join(environ.get('OUT_PATH'), 'ADNI/OUTPUT')
spie_output_dir = path.join(adni_output_dir, 'SPIE')

scores_dir = path.join(spie_output_dir, 'input_data/svm_scores')
output_path = path.join(spie_output_dir, 'input_data')


### Joining T1 and FDG-PET scores

T1_df = pd.io.parsers.read_csv(path.join(scores_dir, 'T1.tsv'), sep='\t')
FDG_df = pd.io.parsers.read_csv(path.join(scores_dir, 'fdg.tsv'), sep='\t')
joined = T1_df.merge(FDG_df, on=['diagnosis', 'participant_id', 'session_id'])
joined = joined.rename(columns={'y_hat_x': 'T1_score', 'y_hat_y': 'fdg_score'})
subj_sessions = joined[['participant_id', 'session_id', 'diagnosis', 'T1_score', 'fdg_score']]

participant_columns = ["sex", "education_level", "marital_status", "apoe4", "apoe_gen1", "apoe_gen2"]
session_columns = ["age",
                   # Cognitive measures
                   "MMSE", "cdr_sb", "cdr_global", "adas11", "adas13",
                   "adas_memory", "adas_language", "adas_concentration", "adas_praxis", "ravlt_immediate", "moca",
                   "TMT_A", "TMT_B", "dsst", "logmem_delay", "logmem_imm",
                   # T1 measures
                   "adni_ventricles_vol", "adni_hippocampus_vol", "adni_brain_vol", "adni_entorhinal_vol",
                   "adni_fusiform_vol", "adni_midtemp_vol", "adni_icv",
                   # PET measures
                   "adni_fdg", "adni_pib", "adni_av45",
                   # CSF measures
                   "adni_abeta", "adni_tau", "adni_ptau"]

participant_series = {}
session_series = {}
for col in participant_columns:
    participant_series[col] = []
for col in session_columns:
    session_series[col] = []

participants_tsv = pd.io.parsers.read_csv(path.join(adni_bids, "participants.tsv"), sep='\t')

for row in subj_sessions.iterrows():
    subj_sess = row[1]
    selected_participant = participants_tsv[(participants_tsv.participant_id == subj_sess.participant_id)].iloc[0]
    for col in participant_columns:
        participant_series[col].append(selected_participant[col])

    session_tsv = pd.io.parsers.read_csv(path.join(adni_bids, subj_sess.participant_id,
                                                   subj_sess.participant_id + "_sessions.tsv"), sep='\t')
    selected_session = session_tsv[(session_tsv.session_id == subj_sess.session_id)].iloc[0]
    for col in session_columns:
        session_series[col].append(selected_session[col])

for col in participant_columns:
    subj_sessions.loc[:, col] = pd.Series(participant_series[col], index=subj_sessions.index)

for col in session_columns:
    subj_sessions.loc[:, col] = pd.Series(session_series[col], index=subj_sessions.index)

subj_sessions.to_csv(path.join(output_path, 'input_data.tsv'), sep='\t', index=False)


### Generating different data tables

For each of the the different populations determined by data availability, a separated data input file is created and splits for cross-validation are generated.
(from `data_selection/generate_input_data.py`)

In [ ]:
from os import path, environ

import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

from workflow import TsvInput


adni_output_dir = path.join(environ.get('OUT_PATH'), 'ADNI/OUTPUT')
spie_output_dir = path.join(adni_output_dir, 'SPIE')

input_data_path = path.join(spie_output_dir, 'input_data')
tasks_data_path = path.join(spie_output_dir, 'tasks_lists')

subj_sessions = pd.io.parsers.read_csv(path.join(input_data_path, 'input_data.tsv'), sep='\t')

subj_sessions.loc[subj_sessions[subj_sessions.sex == 'F'].index, 'sex'] = 1
subj_sessions.loc[subj_sessions[subj_sessions.sex == 'M'].index, 'sex'] = 0


##################################################
### Table 1 - Clinical data and Imaging scores ###
##################################################

model_36 = subj_sessions[~subj_sessions.adas13.isnull()]
model_36.to_csv(path.join(input_data_path, 'table_1_clinical_imaging', 'input_data_model_36.tsv'), sep='\t', index=False)


####################################################
### Table 2 - Clinical data and ADNIMERGE scores ###
####################################################

model_36 = subj_sessions[~subj_sessions.adas13.isnull() &
                         ~subj_sessions.adni_ventricles_vol.isnull() &
                         ~subj_sessions.adni_hippocampus_vol.isnull() &
                         ~subj_sessions.adni_brain_vol.isnull() &
                         ~subj_sessions.adni_entorhinal_vol.isnull() &
                         ~subj_sessions.adni_icv.isnull() &
                         ~subj_sessions.adni_fusiform_vol.isnull() &
                         ~subj_sessions.adni_midtemp_vol.isnull()]

model_36["adni_ventricles_vol_icv"] = model_36.adni_ventricles_vol / model_36.adni_icv
model_36["adni_hippocampus_vol_icv"] = model_36.adni_hippocampus_vol / model_36.adni_icv
model_36["adni_brain_vol_icv"] = model_36.adni_brain_vol / model_36.adni_icv
model_36["adni_entorhinal_vol_icv"] = model_36.adni_entorhinal_vol / model_36.adni_icv
model_36["adni_fusiform_vol_icv"] = model_36.adni_fusiform_vol / model_36.adni_icv
model_36["adni_midtemp_vol_icv"] = model_36.adni_midtemp_vol / model_36.adni_icv

model_36.to_csv(path.join(input_data_path, 'table_2_clinical_adnimerge', 'input_data_model_36.tsv'), sep='\t', index=False)


######################################################
### Table 3 - Clinical data and Amyloid deposition ###
######################################################

model_36 = subj_sessions[~subj_sessions.adas13.isnull() &
                         (~subj_sessions.adni_pib.isnull() |
                          ~subj_sessions.adni_av45.isnull())]

model_36["amyloid"] = ((model_36.adni_pib >= 1.47) * 1.0) + ((model_36.adni_av45 >= 1.1) * 1.0)

model_36.to_csv(path.join(input_data_path, 'table_3_amyloid', 'input_data_model_36.tsv'), sep='\t', index=False)


####################################
### Table 4 - Several timepoints ###
####################################

model_36 = subj_sessions[~subj_sessions.adas13.isnull()]

for i in range(1, 7):

    model = []
    diagnoses = pd.io.parsers.read_csv(path.join(tasks_data_path, 'sMCI_vs_pMCI_diagnoses_%s.tsv' % (i * 6)), sep='\t')

    for row in diagnoses.iterrows():
        subj = row[1]
        data_subj = model_36[model_36.participant_id == subj.participant_id]
        if data_subj.shape[0] == 1:
            data_subj.diagnosis = 0
            data_subj.diagnosis = subj.diagnosis
            model.append(data_subj)

    model_t = pd.concat(model)
    model_t.to_csv(path.join(input_data_path, 'table_4_timepoints', 'input_data_model_%s.tsv' % (i * 6)), sep='\t', index=False)


########################################
### Generating splits for each table ###
########################################

n_iterations = 250
test_size = 0.2

tables_timepoints = {'table_1_clinical_imaging': [36],
                     'table_2_clinical_adnimerge': [36],
                     'table_3_amyloid': [36],
                     'table_4_timepoints': [6, 12, 18, 24, 30, 36]}

for table in tables_timepoints:
    for timepoint in tables_timepoints[table]:

        data_tsv = path.join(input_data_path, table, 'input_data_model_%s.tsv' % timepoint)
        input_data = TsvInput(data_tsv)
        y = input_data.get_y()

        splits = StratifiedShuffleSplit(n_splits=n_iterations, test_size=test_size)
        splits_indices = list(splits.split(np.zeros(len(y)), y))

        with open(path.join(input_data_path, table, 'indices_model_%s.tsv' % timepoint), 'wb') as s:
            pickle.dump(splits_indices, s)


## 7. Experiments using demographic, clinical data, and imaging scores

(from `experiments/1_clinical_imaging/rf_clinical_imaging.py`)

In [ ]:

import os
import sys
import inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

from general_models import rf_classifications

# Paths to personalize
spie_output_dir = os.path.join(os.environ.get('OUT_PATH'), 'ADNI/OUTPUT', 'SPIE')

input_data_path = os.path.join(spie_output_dir, 'input_data', 'table_1_clinical_imaging')
data_tsv_template = input_data_path + "/input_data_model_%s.tsv"
indices_template = input_data_path + "/indices_model_%s.pkl"
output_dir = os.path.join(spie_output_dir, 'output_data', '1_clinical_imaging')

n_threads = 8
months = [36]

models = {

          # Models using only demographic and clinical data

          "base": ["sex", "education_level", "MMSE", "cdr_sb"],

          "base_logmem": ["sex", "education_level", "MMSE", "cdr_sb", "logmem_delay", "logmem_imm"],

          "base_ravlt": ["sex", "education_level", "MMSE", "cdr_sb", "ravlt_immediate"],

          "base_logmem_ravlt": ["sex", "education_level", "MMSE", "cdr_sb", "ravlt_immediate", "logmem_delay",
                                "logmem_imm"],

          "base_adas": ["sex", "education_level", "MMSE", "cdr_sb", "adas_memory", "adas_language",
                              "adas_concentration", "adas_praxis"],

          "base_ravlt_adas": ["sex", "education_level", "MMSE", "cdr_sb", "adas_memory", "adas_language",
                              "adas_concentration", "adas_praxis", "ravlt_immediate"],

          # Models including APOE

          "base_ravlt_apoe": ["sex", "education_level", "apoe4", "MMSE", "cdr_sb", "ravlt_immediate"],

          "base_adas_apoe": ["sex", "education_level", "apoe4", "MMSE", "cdr_sb", "adas_memory", "adas_language",
                             "adas_concentration", "adas_praxis"],

          "base_ravlt_adas_apoe": ["sex", "education_level", "apoe4", "MMSE", "cdr_sb", "adas_memory", "adas_language",
                                   "adas_concentration", "adas_praxis", "ravlt_immediate"],

          # Models including imaging scores

          "base_T1score": ["sex", "education_level", "MMSE", "cdr_sb", "T1_score"],

          "base_fdgscore": ["sex", "education_level", "MMSE", "cdr_sb", "fdg_score"],

          "base_scores": ["sex", "education_level", "MMSE", "cdr_sb", "T1_score", "fdg_score"],

          "base_ravlt_scores": ["sex", "education_level", "MMSE", "cdr_sb", "ravlt_immediate", "T1_score", "fdg_score"],

          "base_adas_scores": ["sex", "education_level", "MMSE", "cdr_sb", "adas_memory", "adas_language",
                               "adas_concentration", "adas_praxis", "T1_score", "fdg_score"],

          "base_adas_memtest_scores": ["sex", "education_level", "MMSE", "cdr_sb", "adas_memory", "adas_language",
                                       "adas_concentration", "adas_praxis", "ravlt_immediate", "T1_score", "fdg_score"]

}

for model_name in models:
    rf_classifications(model_name, models[model_name], data_tsv_template, indices_template, output_dir, months,
                       n_threads=n_threads)


## Other experiments

Other classification tasks can be run directly from the `experiments` directory:

####  Using demographic, clinical data, and imaging data from ADNIMERGE file

```
python experiments/2_clinical_adnimerge/rf_clinical_adnimerge.py
```

####  Taking subjects amyloid status into account

```
python experiments/3_amyloid/rf_amyloid.py
```

#### Predicting progression at different time-points

```
python experiments/3_amyloid/rf_amyloid.py
```
